<a href="https://colab.research.google.com/github/Michel-p16/PDS-Project/blob/Michel/text_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
# Install necessary libraries
!pip install transformers pandas
!pip install rouge_score
!pip install bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.8 MB/s eta 0:00:00


In [20]:
import json
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [14]:
from google.colab import files

uploaded = files.upload()

# Load the JSON file
file_name = list(uploaded.keys())[0]
with open(file_name, 'r') as f:
    data = json.load(f)

Saving generated_responses_few_errors.json to generated_responses_few_errors (1).json


In [15]:
# Initialize T5 model and tokenizer
model_name = "t5-small"  # You can use t5-base or t5-large for better results
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [16]:

# Function to summarize text using T5
def summarize_text(text, max_length=50):
    input_text = f"summarize: {text}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = model.generate(input_ids, max_length=max_length, min_length=10, length_penalty=2.0, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [17]:
# Extract and summarize answers of type 'TEXT'
summarized_data = []
for entry in data:
    if entry.get("type") == "TEXT":
        for answer in entry.get("answers", []):
            if "text" in answer and answer["text"].strip():
                summarized_answer = summarize_text(answer["text"])
                summarized_data.append({
                    "question_id": entry["question_id"],
                    "original_text": answer["text"],
                    "summarized_text": summarized_answer,
                    "timestamp": answer.get("timestamp")
                })

In [18]:
# Convert summarized data to a DataFrame and display it
df_summarized = pd.DataFrame(summarized_data)

# Save the summarized data to a CSV file
df_summarized.to_csv("summarized_responses.csv", index=False)

# Display the DataFrame
from google.colab.data_table import DataTable
DataTable(df_summarized)

,question_id,original_text,summarized_text,timestamp
0,a0148bc7-15b3-41d5-b97c-6420b8bd927c,"Sure, what do you want me to write?",what do you want me to write? what do you want...,2025-01-21T12:11:02.478842
1,a0148bc7-15b3-41d5-b97c-6420b8bd927c,I don't know,"i don't know what I'm doing, but I don't know.",2025-01-21T12:11:02.478889
2,a0148bc7-15b3-41d5-b97c-6420b8bd927c,Sure.,Sure.. Sure.. Sure.,2025-01-21T12:11:08.295011
3,a0148bc7-15b3-41d5-b97c-6420b8bd927c,Sure thing!,Sure thing!.... Sure thing!,2025-01-21T12:11:13.883833
4,a0148bc7-15b3-41d5-b97c-6420b8bd927c,I don't care,i don't care what I'm doing.,2025-01-21T12:11:13.883886
...,...,...,...,...
93,a0148bc7-15b3-41d5-b97c-6420b8bd927c,I don't know,"i don't know what I'm doing, but I don't know.",2025-01-21T12:18:27.560255
94,a0148bc7-15b3-41d5-b97c-6420b8bd927c,"Sure, what do you need?",what do you need? what do you need?,2025-01-21T12:18:33.474813
95,a0148bc7-15b3-41d5-b97c-6420b8bd927c,"Sure, what's up?","what's up? Sure, what's up?",2025-01-21T12:18:39.114890
96,a0148bc7-15b3-41d5-b97c-6420b8bd927c,Sure thing!,Sure thing!.... Sure thing!,2025-01-21T12:18:44.804798


In [21]:
from rouge_score import rouge_scorer

def calculate_rouge_scores(references, summaries):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = []
    for ref, summ in zip(references, summaries):
        score = scorer.score(ref, summ)
        scores.append({
            "ROUGE-1": score['rouge1'].fmeasure,
            "ROUGE-2": score['rouge2'].fmeasure,
            "ROUGE-L": score['rougeL'].fmeasure
        })
    return scores

In [23]:
# Beispiel: Vergleiche Originaltexte mit zusammengefassten Texten
references = df_summarized["original_text"]
summaries = df_summarized["summarized_text"]

rouge_scores = calculate_rouge_scores(references, summaries)

# Füge die ROUGE-Ergebnisse zur DataFrame hinzu
df_summarized["ROUGE-1"] = [score["ROUGE-1"] for score in rouge_scores]
df_summarized["ROUGE-2"] = [score["ROUGE-2"] for score in rouge_scores]
df_summarized["ROUGE-L"] = [score["ROUGE-L"] for score in rouge_scores]

# Ergebnisse speichern
df_summarized.to_csv("summarized_responses_with_rouge.csv", index=False)

# Anzeige der Ergebnisse
DataTable(df_summarized)

,question_id,original_text,summarized_text,timestamp,ROUGE-1,ROUGE-2,ROUGE-L
0,a0148bc7-15b3-41d5-b97c-6420b8bd927c,"Sure, what do you want me to write?",what do you want me to write? what do you want...,2025-01-21T12:11:02.478842,0.666667,0.631579,0.666667
1,a0148bc7-15b3-41d5-b97c-6420b8bd927c,I don't know,"i don't know what I'm doing, but I don't know.",2025-01-21T12:11:02.478889,0.470588,0.400000,0.470588
2,a0148bc7-15b3-41d5-b97c-6420b8bd927c,Sure.,Sure.. Sure.. Sure.,2025-01-21T12:11:08.295011,0.500000,0.000000,0.500000
3,a0148bc7-15b3-41d5-b97c-6420b8bd927c,Sure thing!,Sure thing!.... Sure thing!,2025-01-21T12:11:13.883833,0.666667,0.500000,0.666667
4,a0148bc7-15b3-41d5-b97c-6420b8bd927c,I don't care,i don't care what I'm doing.,2025-01-21T12:11:13.883886,0.666667,0.600000,0.666667
...,...,...,...,...,...,...,...
93,a0148bc7-15b3-41d5-b97c-6420b8bd927c,I don't know,"i don't know what I'm doing, but I don't know.",2025-01-21T12:18:27.560255,0.470588,0.400000,0.470588
94,a0148bc7-15b3-41d5-b97c-6420b8bd927c,"Sure, what do you need?",what do you need? what do you need?,2025-01-21T12:18:33.474813,0.615385,0.545455,0.615385
95,a0148bc7-15b3-41d5-b97c-6420b8bd927c,"Sure, what's up?","what's up? Sure, what's up?",2025-01-21T12:18:39.114890,0.727273,0.666667,0.727273
96,a0148bc7-15b3-41d5-b97c-6420b8bd927c,Sure thing!,Sure thing!.... Sure thing!,2025-01-21T12:18:44.804798,0.666667,0.500000,0.666667


In [25]:
from bert_score import score as bert_score

def calculate_bert_scores(references, summaries):
    P, R, F1 = bert_score(summaries, references, lang="en", model_type="bert-base-uncased")
    return list(F1.numpy())

# Berechne BERTScore
bert_scores = calculate_bert_scores(references.tolist(), summaries.tolist())

# BERTScore zur DataFrame hinzufügen
df_summarized["BERTScore"] = bert_scores

# Ergebnisse speichern
df_summarized.to_csv("summarized_responses_with_bertscore.csv", index=False)

# Anzeige der Ergebnisse
df_summarized.head()

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

,question_id,original_text,summarized_text,timestamp,ROUGE-1,ROUGE-2,ROUGE-L,BERTScore
0,a0148bc7-15b3-41d5-b97c-6420b8bd927c,"Sure, what do you want me to write?",what do you want me to write? what do you want...,2025-01-21T12:11:02.478842,0.666667,0.631579,0.666667,0.794010
1,a0148bc7-15b3-41d5-b97c-6420b8bd927c,I don't know,"i don't know what I'm doing, but I don't know.",2025-01-21T12:11:02.478889,0.470588,0.400000,0.470588,0.704476
2,a0148bc7-15b3-41d5-b97c-6420b8bd927c,Sure.,Sure.. Sure.. Sure.,2025-01-21T12:11:08.295011,0.500000,0.000000,0.500000,0.658947
3,a0148bc7-15b3-41d5-b97c-6420b8bd927c,Sure thing!,Sure thing!.... Sure thing!,2025-01-21T12:11:13.883833,0.666667,0.500000,0.666667,0.658054
4,a0148bc7-15b3-41d5-b97c-6420b8bd927c,I don't care,i don't care what I'm doing.,2025-01-21T12:11:13.883886,0.666667,0.600000,0.666667,0.757795
